In [10]:
import glob
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import re
from decimal import Decimal
from datetime import datetime
import pandas_gbq
pd.set_option('display.max_columns', None)  

In [11]:
# current_date = datetime.now()
# first_day = current_date.replace(day=1)
# formatted_date = first_day.strftime("%Y%m%d")
# formatted_date

In [12]:
folder_main = "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/"
division_lst = ['CPD','LDB','LLD','PPD']
# division_lst = ['LDB']
date_formatted_input = 20231201

## 1.  Total Order

### 1.A. Read all files from folder

In [13]:
csv_files_raw = []
folder_lst = []

for division in division_lst:
    folder_path = folder_main + division
    folder_lst.append(folder_path)
for folder in folder_lst:
    folder_order = folder + "/Orders/"
    csv_files_lst = glob.glob(folder_order + "\*.xlsx")
    for csv_file in csv_files_lst:
        csv_files_raw.append(csv_file)

In [14]:
csv_files = []

def extract_month(folder_location_string):
    date_extract_pattern = r'\d{4}_\d{2}_\d{2}'
    match = re.search(date_extract_pattern, folder_location_string)
    if match:
        extracted_date = match.group()
        date_formatted = int(extracted_date.replace("_", ""))
        if date_formatted >= date_formatted_input:
            csv_files.append(folder_location_string)  
    
for i in csv_files_raw:
    extract_month(i)   
    
print('Total files: ' + str(len(csv_files)))    
csv_files

Total files: 5


["C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/CPD/Orders\\mny_tiktok_orders_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/CPD/Orders\\oap_tiktok_orders_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LDB/Orders\\cerave_tiktok_orders_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LDB/Orders\\derma_tiktok_orders_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LLD/Orders\\kiehls_tiktok_orders_2023_12_01.xlsx"]

### 1.B. Append all csv files into a dataframe

In [15]:
dfs = []
for file in csv_files:
    df = pd.read_excel(file, dtype={
        'Seller Note': str
        ,'Checked Marked by': str
        ,'additional_address_information': str
    })
    
    if re.search("cerave", str(file), re.IGNORECASE):
        df['brand'] = 'CeraVe'
    elif re.search("kiehls", str(file), re.IGNORECASE):
        df['brand'] = "Kiehl's"
    elif re.search("oap", str(file), re.IGNORECASE):
        df['brand'] = "L'Oreal Paris"
    elif re.search("mny", str(file), re.IGNORECASE):
        df['brand'] = "Maybelline"
    elif re.search("derma", str(file), re.IGNORECASE):
        df['brand'] = "Derma"
        
    dfs.append(df)

df_order = pd.concat(dfs)    
df_order.rename(columns={'Order ID': 'order_id', 'Order Status': 'order_status', 'Order Substatus': 'order_substatus', 'Cancelation/Return Type': 'cancelation_type', 'Normal or Pre-order': 'normal_or_preorder', 'SKU ID': 'sku_id', 'Seller SKU': 'seller_sku', 'Product Name': 'product_name', 'Variation': 'variation', 'Quantity': 'quantity', 'Sku Quantity of return': 'sku_quantity_of_return', 'SKU Unit Original Price': 'sku_unit_original_price', 'SKU Subtotal Before Discount': 'sku_subtotal_before_discount', 'SKU Platform Discount': 'sku_platform_discount', 'SKU Seller Discount': 'sku_seller_discount', 'SKU Subtotal After Discount': 'sku_subtotal_after_discount', 'Shipping Fee After Discount': 'shipping_fee_after_discount', 'Original Shipping Fee': 'original_shipping_fee', 'Shipping Fee Seller Discount': 'shipping_fee_seller_discount', 'Shipping Fee Platform Discount': 'shipping_fee_platform_discount', 'Taxes': 'taxes', 'Order Amount': 'order_amount', 'Order Refund Amount': 'order_refund_amount', 'Created Time': 'created_time', 'Paid Time': 'paid_time', 'RTS Time': 'rts_time', 'Shipped Time': 'shipped_time', 'Delivered Time': 'delivered_time', 'Cancelled Time': 'cancelled_time', 'Cancel By': 'cancel_by', 'Cancel Reason': 'cancel_reason', 'Fulfillment Type': 'fulfillment_type', 'Warehouse Name': 'warehouse_name', 'Tracking ID': 'tracking_id', 'Delivery Option': 'delivery_option', 'Shipping Provider Name': 'shipping_provider_name', 'Buyer Message': 'buyer_message', 'Buyer Username': 'buyer_username', 'Recipient': 'recipient', 'Phone #': 'phone', 'Country': 'country', 'Province': 'province', 'District': 'district', 'Commune': 'commune', 'Detail Address': 'detail_address', 'Additional address information': 'additional_address_information', 'Payment Method': 'payment_method', 'Weight(kg)': 'weight', 'Product Category': 'product_category', 'Package ID': 'package_id', 'Seller Note': 'seller_note', 'Checked Status': 'checked_status', 'Checked Marked by': 'checked_marked_by'
                        }, errors="raise", inplace=True)
df_order = df_order.dropna(axis=0, subset=['order_id'])
# # df_order.set_index('order_id', inplace=True)
# print(len(df_order))
print(len(df_order))
df_order.head(1)

125011


,order_id,order_status,order_substatus,cancelation_type,normal_or_preorder,sku_id,seller_sku,product_name,variation,quantity,sku_quantity_of_return,sku_unit_original_price,sku_subtotal_before_discount,sku_platform_discount,sku_seller_discount,sku_subtotal_after_discount,shipping_fee_after_discount,original_shipping_fee,shipping_fee_seller_discount,shipping_fee_platform_discount,Payment platform discount,taxes,order_amount,order_refund_amount,created_time,paid_time,rts_time,shipped_time,delivered_time,cancelled_time,cancel_by,cancel_reason,fulfillment_type,warehouse_name,tracking_id,delivery_option,shipping_provider_name,buyer_message,buyer_username,recipient,phone,country,province,district,commune,detail_address,additional_address_information,payment_method,weight,product_category,package_id,seller_note,checked_status,checked_marked_by,brand
0,578249711791212696,Shipped,In Transit,NaN,Normal,1729799605019904186,TUML01068CB,[Bộ trang điểm] Kem Nền Kiềm Dầu Fit Me 30ml v...,FM FDT 110 + CP 109,1,0,446.000 VND,446.000 VND,0 VND,58.000 VND,388.000 VND,8.100 VND,33.100 VND,0 VND,25.000 VND,NaN,0 VND,396.100 VND,NaN,18/12/2023 23:59:11\t,18/12/2023 23:59:26\t,19/12/2023 00:03:26\t,19/12/2023 10:36:33\t,\t,\t,NaN,NaN,Fulfillment by seller,Kho Tiki Tân Bình,855825474604.0,Standard shipping,J&T Express,NaN,atham1105,N***** T** A T***,(+84)092*****86,Việt Nam,Bình Dương,Thủ Dầu Một,Ph* **,Nă* *** *** *** ***** *** **** *** *** *** ***...,NaN,Momo,0.163,Makeup Sets,1153876103991167128\t,NaN,Unchecked,NaN,Maybelline


### 1.C. Clean the order dataframe

In [16]:
def clean_vnd(x): 
    return float(str(x).replace('VND','').replace('.','').replace(' ',''))

# def clean_time(x): 
#     return str(x).replace('\t','').strip()

def clean_time(x): 
    time_string = str(x).replace('\t','').strip()
    try:
        return datetime.strptime(time_string, '%d/%m/%Y %H:%M:%S')
    except:
        return None

def clean_order_id(x): 
    return str(x).replace('\t','')

def clean_sku_id(x): 
    return str(x).replace('\t','')

df_order['sku_unit_original_price'] = df_order['sku_unit_original_price'].apply(clean_vnd)   
df_order['sku_subtotal_before_discount'] = df_order['sku_subtotal_before_discount'].apply(clean_vnd)
df_order['sku_platform_discount'] = df_order['sku_platform_discount'].apply(clean_vnd)
df_order['sku_seller_discount'] = df_order['sku_seller_discount'].apply(clean_vnd)
df_order['sku_subtotal_after_discount'] = df_order['sku_subtotal_after_discount'].apply(clean_vnd)
df_order['shipping_fee_after_discount'] = df_order['shipping_fee_after_discount'].apply(clean_vnd)
df_order['original_shipping_fee'] = df_order['original_shipping_fee'].apply(clean_vnd)
df_order['shipping_fee_seller_discount'] = df_order['shipping_fee_seller_discount'].apply(clean_vnd)
df_order['shipping_fee_platform_discount'] = df_order['shipping_fee_platform_discount'].apply(clean_vnd)
df_order['taxes'] = df_order['taxes'].apply(clean_vnd)
df_order['order_amount'] = df_order['order_amount'].apply(clean_vnd)
df_order['order_refund_amount'] = df_order['order_refund_amount'].apply(clean_vnd)
df_order['tracking_id'] = df_order['tracking_id'].astype(str)

df_order['created_time'] = df_order['created_time'].apply(clean_time)
df_order['paid_time'] = df_order['paid_time'].apply(clean_time)
df_order['rts_time'] = df_order['rts_time'].apply(clean_time)
df_order['shipped_time'] = df_order['shipped_time'].apply(clean_time)
df_order['delivered_time'] = df_order['delivered_time'].apply(clean_time)
df_order['cancelled_time'] = df_order['cancelled_time'].apply(clean_time)

# df_order['created_time'] = pd.to_datetime(df_order['created_time'].apply(clean_time), format='%d/%m/%Y %H:%M:%S', errors = 'ignore')
# df_order['paid_time'] = pd.to_datetime(df_order['paid_time'].apply(clean_time), format='%d/%m/%Y %H:%M:%S', errors = 'ignore')
# df_order['rts_time'] = pd.to_datetime(df_order['rts_time'].apply(clean_time), format='%d/%m/%Y %H:%M:%S', errors = 'ignore')
# df_order['shipped_time'] = pd.to_datetime(df_order['shipped_time'].apply(clean_time), format='%d/%m/%Y %H:%M:%S', errors = 'ignore')
# df_order['delivered_time'] = pd.to_datetime(df_order['delivered_time'].apply(clean_time), format='%d/%m/%Y %H:%M:%S', errors = 'ignore')
# df_order['cancelled_time'] = pd.to_datetime(df_order['cancelled_time'].apply(clean_time), format='%d/%m/%Y %H:%M:%S', errors = 'ignore')

df_order['order_id'] = df_order['order_id'].apply(clean_order_id)
df_order['sku_id'] = df_order['sku_id'].apply(clean_order_id)

df_order['product_name'] = df_order['product_name'].astype(str)
df_order['variation'] = df_order['variation'].astype(str)
df_order['cancel_by'] = df_order['cancel_by'].astype(str)
df_order['cancel_reason'] = df_order['cancel_reason'].astype(str)
df_order['fulfillment_type'] = df_order['fulfillment_type'].astype(str)
df_order['warehouse_name'] = df_order['warehouse_name'].astype(str)
df_order['tracking_id'] = df_order['tracking_id'].astype(str)
df_order['delivery_option'] = df_order['delivery_option'].astype(str)
df_order['shipping_provider_name'] = df_order['shipping_provider_name'].astype(str)
df_order['buyer_message'] = df_order['buyer_message'].astype(str)
df_order['buyer_username'] = df_order['buyer_username'].astype(str)
df_order['recipient'] = df_order['recipient'].astype(str)
df_order['phone'] = df_order['phone'].astype(str)
df_order['country'] = df_order['country'].astype(str)
df_order['province'] = df_order['province'].astype(str)
df_order['district'] = df_order['district'].astype(str)
df_order['commune'] = df_order['commune'].astype(str)
df_order['detail_address'] = df_order['detail_address'].astype(str)
df_order['additional_address_information'] = df_order['additional_address_information'].astype(str)
df_order['payment_method'] = df_order['payment_method'].astype(str)
df_order['product_category'] = df_order['product_category'].astype(str)
df_order['package_id'] = df_order['package_id'].astype(str)
df_order['seller_note'] = df_order['seller_note'].astype(str)
df_order['checked_status'] = df_order['checked_status'].astype(str)
df_order['checked_marked_by'] = df_order['checked_marked_by'].astype(str)
df_order['brand'] = df_order['brand'].astype(str)

df_order.set_index('order_id', inplace=True)
df_order = df_order.drop_duplicates()
print(len(df_order))
df_order.head(1)

125011


,order_status,order_substatus,cancelation_type,normal_or_preorder,sku_id,seller_sku,product_name,variation,quantity,sku_quantity_of_return,sku_unit_original_price,sku_subtotal_before_discount,sku_platform_discount,sku_seller_discount,sku_subtotal_after_discount,shipping_fee_after_discount,original_shipping_fee,shipping_fee_seller_discount,shipping_fee_platform_discount,Payment platform discount,taxes,order_amount,order_refund_amount,created_time,paid_time,rts_time,shipped_time,delivered_time,cancelled_time,cancel_by,cancel_reason,fulfillment_type,warehouse_name,tracking_id,delivery_option,shipping_provider_name,buyer_message,buyer_username,recipient,phone,country,province,district,commune,detail_address,additional_address_information,payment_method,weight,product_category,package_id,seller_note,checked_status,checked_marked_by,brand
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
578249711791212696,Shipped,In Transit,NaN,Normal,1729799605019904186,TUML01068CB,[Bộ trang điểm] Kem Nền Kiềm Dầu Fit Me 30ml v...,FM FDT 110 + CP 109,1,0,446000.0,446000.0,0.0,58000.0,388000.0,8100.0,33100.0,0.0,25000.0,NaN,0.0,396100.0,NaN,2023-12-18 23:59:11,2023-12-18 23:59:26,2023-12-19 00:03:26,2023-12-19 10:36:33,NaT,NaT,nan,nan,Fulfillment by seller,Kho Tiki Tân Bình,855825474604.0,Standard shipping,J&T Express,nan,atham1105,N***** T** A T***,(+84)092*****86,Việt Nam,Bình Dương,Thủ Dầu Một,Ph* **,Nă* *** *** *** ***** *** **** *** *** *** ***...,nan,Momo,0.163,Makeup Sets,1153876103991167128\t,nan,Unchecked,nan,Maybelline


### 1.D. Write order to GCP

A. Create a new table

In [17]:
import pandas
import pandas_gbq

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "spmena-sandbox-vn-apac-dv"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'tiktok_dashboard.tiktok_order_intraday'

pandas_gbq.to_gbq(df_order, table_id, project_id=project_id, if_exists = 'replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


B. Merge to full table

In [18]:
sql = """
MERGE INTO `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_order` t1
USING `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_order_intraday` t2
ON t1.order_id = t2.order_id and t1.sku_id = t2.sku_id
when matched then
  update
  set t1.order_status = t2.order_status, t1.order_substatus = t2.order_substatus, t1.cancelation_type = t2.cancelation_type, t1.normal_or_preorder = t2.normal_or_preorder, t1.seller_sku = t2.seller_sku, t1.product_name = t2.product_name, t1.variation = t2.variation, t1.quantity = t2.quantity, t1.sku_quantity_of_return = t2.sku_quantity_of_return, t1.sku_unit_original_price = t2.sku_unit_original_price, t1.sku_subtotal_before_discount = t2.sku_subtotal_before_discount, t1.sku_platform_discount = t2.sku_platform_discount, t1.sku_seller_discount = t2.sku_seller_discount, t1.sku_subtotal_after_discount = t2.sku_subtotal_after_discount, t1.shipping_fee_after_discount = t2.shipping_fee_after_discount, t1.original_shipping_fee = t2.original_shipping_fee, t1.shipping_fee_seller_discount = t2.shipping_fee_seller_discount, t1.shipping_fee_platform_discount = t2.shipping_fee_platform_discount, t1.taxes = t2.taxes, t1.order_amount = t2.order_amount, t1.order_refund_amount = t2.order_refund_amount, t1.created_time = t2.created_time, t1.paid_time = t2.paid_time, t1.rts_time = t2.rts_time, t1.shipped_time = t2.shipped_time, t1.delivered_time = t2.delivered_time, t1.cancelled_time = t2.cancelled_time, t1.cancel_by = t2.cancel_by, t1.cancel_reason = t2.cancel_reason, t1.fulfillment_type = t2.fulfillment_type, t1.warehouse_name = t2.warehouse_name, t1.tracking_id = t2.tracking_id, t1.delivery_option = t2.delivery_option, t1.shipping_provider_name = t2.shipping_provider_name, t1.buyer_message = t2.buyer_message, t1.buyer_username = t2.buyer_username, t1.recipient = t2.recipient, t1.phone = t2.phone, t1.country = t2.country, t1.province = t2.province, t1.district = t2.district, t1.commune = t2.commune, t1.detail_address = t2.detail_address, t1.additional_address_information = t2.additional_address_information, t1.payment_method = t2.payment_method, t1.weight = t2.weight, t1.product_category = t2.product_category, t1.package_id = t2.package_id, t1.seller_note = t2.seller_note, t1.checked_status = t2.checked_status, t1.checked_marked_by = t2.checked_marked_by, t1.brand = t2.brand
WHEN NOT MATCHED THEN
  INSERT (order_id, order_status, order_substatus, cancelation_type, normal_or_preorder, sku_id, seller_sku, product_name, variation, quantity, sku_quantity_of_return, sku_unit_original_price, sku_subtotal_before_discount, sku_platform_discount, sku_seller_discount, sku_subtotal_after_discount, shipping_fee_after_discount, original_shipping_fee, shipping_fee_seller_discount, shipping_fee_platform_discount, taxes, order_amount, order_refund_amount, created_time, paid_time, rts_time, shipped_time, delivered_time, cancelled_time, cancel_by, cancel_reason, fulfillment_type, warehouse_name, tracking_id, delivery_option, shipping_provider_name, buyer_message, buyer_username, recipient, phone, country, province, district, commune, detail_address, additional_address_information, payment_method, weight, product_category, package_id, seller_note, checked_status, checked_marked_by, brand)
  VALUES (order_id, order_status, order_substatus, cancelation_type, normal_or_preorder, sku_id, seller_sku, product_name, variation, quantity, sku_quantity_of_return, sku_unit_original_price, sku_subtotal_before_discount, sku_platform_discount, sku_seller_discount, sku_subtotal_after_discount, shipping_fee_after_discount, original_shipping_fee, shipping_fee_seller_discount, shipping_fee_platform_discount, taxes, order_amount, order_refund_amount, created_time, paid_time, rts_time, shipped_time, delivered_time, cancelled_time, cancel_by, cancel_reason, fulfillment_type, warehouse_name, tracking_id, delivery_option, shipping_provider_name, buyer_message, buyer_username, recipient, phone, country, province, district, commune, detail_address, additional_address_information, payment_method, weight, product_category, package_id, seller_note, checked_status, checked_marked_by, brand);
"""
project_id = "spmena-sandbox-vn-apac-dv"
pd.read_gbq(sql,project_id=project_id, use_bqstorage_api=True)  
print('Done merging')

Done merging


----------------------------------------------------------------------------------------------------------------------------

## 2. Affiliate folder

### 2.A. Read all csv file from affiliate folder

In [19]:
csv_files_raw = []
folder_lst = []

for division in division_lst:
    folder_path = folder_main + division
    folder_lst.append(folder_path)
for folder in folder_lst:
    folder_order = folder + "/Affiliate Orders/"
    csv_files_lst = glob.glob(folder_order + "\*.xlsx")
    for csv_file in csv_files_lst:
        csv_files_raw.append(csv_file)

In [20]:
csv_files = []

def extract_month(folder_location_string):
    date_extract_pattern = r'\d{4}_\d{2}_\d{2}'
    match = re.search(date_extract_pattern, folder_location_string)
    if match:
        extracted_date = match.group()
        date_formatted = int(extracted_date.replace("_", ""))
        if date_formatted >= date_formatted_input:
            csv_files.append(folder_location_string)  
    
for i in csv_files_raw:
    extract_month(i)   
    
csv_files    

["C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/CPD/Affiliate Orders\\oap_tiktok_affiliate_orders_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LDB/Affiliate Orders\\cerave_tiktok_affiliate_orders_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LDB/Affiliate Orders\\derma_tiktok_affiliate_orders_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LLD/Affiliate Orders\\kiehls_tiktok_affiliate_orders_2023_12_01.xlsx"]

### 2.B. Read all csv file from affiliate folder

In [21]:
dfs = []

for file in csv_files:
    df = pd.read_excel(file, dtype={
            'Content ID': 'object'
            ,'Order ID': 'object'
            ,'Product ID': 'object'
            ,'Sku Id': 'object'
            ,'Price': 'float'
            ,'Payment Amount': 'float'
        })
    
    if re.search("cerave", str(file), re.IGNORECASE):
        df['brand'] = 'CeraVe'
    elif re.search("kiehls", str(file), re.IGNORECASE):
        df['brand'] = "Kiehl's"
    elif re.search("oap", str(file), re.IGNORECASE):
        df['brand'] = "L'Oreal Paris"
    elif re.search("mny", str(file), re.IGNORECASE):
        df['brand'] = "Maybelline"
    elif re.search("derma", str(file), re.IGNORECASE):
        df['brand'] = "Derma"
        
    dfs.append(df)

df_affiliate = pd.concat(dfs)
df_affiliate.rename(columns={'Order ID': 'order_id', 'Product ID': 'product_id', 'Product Name': 'product_name', 'Sku': 'sku_sku', 'Sku Id': 'sku_id', 'Seller Sku': 'seller_sku', 'Price': 'price', 'Payment Amount': 'payment_amount', 'Currency': 'currency', 'Quantity': 'quantity', 'Payment method': 'payment_method', 'Order Status': 'order_status', 'Creator Username': 'creator_username', 'Content Type': 'content_type', 'Content ID': 'content_id', 'Commission Rate (%)': 'commission_rate_percentage', 'Est. Commission Base': 'est_commision_base', 'Est. Commission Payment': 'est_commision_payment', 'Actual Commission Base': 'act_commision_base', 'Actual Commission Payment': 'act_commision_payment', 'Refunds & Returns': 'refunds_returns', 'Refunds': 'refunds', 'Plan ID': 'plan_id', 'Plan Type': 'plan_type', 'Time Created': 'time_created', 'Payment time': 'payment_time', 'Time ready to ship': 'time_ship', 'Order Delivery Time': 'time_delivered', 'Time Order Completed': 'time_completed', 'Time Commission Paid': 'time_commission_paid'}, errors="raise", inplace=True)
df_affiliate.set_index('order_id', inplace=True)
df_affiliate.head(1)

,product_id,product_name,sku_sku,sku_id,seller_sku,price,payment_amount,currency,quantity,payment_method,order_status,creator_username,content_type,content_id,commission_rate_percentage,est_commision_base,est_commision_payment,act_commision_base,act_commision_payment,refunds_returns,refunds,time_created,payment_time,time_ship,time_delivered,time_completed,time_commission_paid,brand,plan_id,plan_type
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
578174226535254351,1729502599715326539,[Mua 2 tặng 2] Kem nền 24h mịn nhẹ lâu trôi L'...,125 Natural Rose,1729502663547849291,6902395719373,289000.0,381200.0,VND,1,Cash on delivery,Cancelled,nguyenttnguyen,Showcase,NaN,6,289000,17340,NaN,NaN,0,0,01/12/2023 00:06:44,NaN,01/12/2023 00:06:45,NaN,NaN,NaN,L'Oreal Paris,NaN,NaN


### 2.C. Clean the affiliate dataframe

In [22]:
df_affiliate['time_created'] = pd.to_datetime(df_affiliate['time_created'], format='%d/%m/%Y %H:%M:%S')
df_affiliate['payment_time'] = pd.to_datetime(df_affiliate['payment_time'], format='%d/%m/%Y %H:%M:%S')
df_affiliate['time_ship'] = pd.to_datetime(df_affiliate['time_ship'], format='%d/%m/%Y %H:%M:%S')
df_affiliate['time_delivered'] = pd.to_datetime(df_affiliate['time_delivered'], format='%d/%m/%Y %H:%M:%S')
df_affiliate['time_completed'] = pd.to_datetime(df_affiliate['time_completed'], format='%d/%m/%Y %H:%M:%S')
df_affiliate['time_commission_paid'] = pd.to_datetime(df_affiliate['time_commission_paid'], format='%d/%m/%Y %H:%M:%S')
df_affiliate['plan_id'] = df_affiliate['plan_id'].fillna(0)
df_affiliate['plan_id'] = df_affiliate['plan_id'].astype(int)
df_affiliate['plan_type'] = df_affiliate['plan_type'].astype(str)
df_affiliate = df_affiliate.drop_duplicates()
print(len(df_affiliate))
df_affiliate.head(2)

55957


,product_id,product_name,sku_sku,sku_id,seller_sku,price,payment_amount,currency,quantity,payment_method,order_status,creator_username,content_type,content_id,commission_rate_percentage,est_commision_base,est_commision_payment,act_commision_base,act_commision_payment,refunds_returns,refunds,time_created,payment_time,time_ship,time_delivered,time_completed,time_commission_paid,brand,plan_id,plan_type
order_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
578174226535254351,1729502599715326539,[Mua 2 tặng 2] Kem nền 24h mịn nhẹ lâu trôi L'...,125 Natural Rose,1729502663547849291,6902395719373,289000.0,381200.0,VND,1,Cash on delivery,Cancelled,nguyenttnguyen,Showcase,NaN,6,289000,17340,NaN,NaN,0,0,2023-12-01 00:06:44,NaT,2023-12-01 00:06:45,NaT,NaT,NaT,L'Oreal Paris,0,nan
578174230967650639,1729502599715326539,[Mua 2 tặng 2] Kem nền 24h mịn nhẹ lâu trôi L'...,125 Natural Rose,1729502663547849291,6902395719373,281000.0,381200.0,VND,1,Cash on delivery,Completed,nguyenttnguyen,Showcase,NaN,6,281000,16860,281000.0,16860.0,0,0,2023-12-01 00:08:21,2023-12-02 10:58:26,2023-12-01 00:08:22,2023-12-02 10:58:26,2023-12-08 10:58:27,2023-12-05 11:22:40,L'Oreal Paris,0,nan


### 2.D. Write affiliate to GCP

A. Create a new table

In [23]:
import pandas
import pandas_gbq

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "spmena-sandbox-vn-apac-dv"

# TODO: Set table_id to the full destination table ID (including the dataset ID).
table_id = 'tiktok_dashboard.tiktok_affiliate_order_intraday'

pandas_gbq.to_gbq(df_affiliate, table_id, project_id=project_id, if_exists = 'replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


B. Merge to full table

In [24]:
sql = """
MERGE INTO `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_affiliate_order` t1
USING `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_affiliate_order_intraday` t2
ON t1.order_id = t2.order_id and t1.sku_id = t2.sku_id
when matched then
  update
  set t1.order_id = t2.order_id, t1.product_id = t2.product_id, t1.product_name = t2.product_name, t1.sku_sku = t2.sku_sku, t1.sku_id = t2.sku_id, t1.seller_sku = t2.seller_sku, t1.price = t2.price, t1.payment_amount = t2.payment_amount, t1.currency = t2.currency, t1.quantity = t2.quantity, t1.payment_method = t2.payment_method, t1.order_status = t2.order_status, t1.creator_username = t2.creator_username, t1.content_type = t2.content_type, t1.content_id = t2.content_id, t1.commission_rate_percentage = t2.commission_rate_percentage, t1.est_commision_base = t2.est_commision_base, t1.est_commision_payment = t2.est_commision_payment, t1.act_commision_base = t2.act_commision_base, t1.act_commision_payment = t2.act_commision_payment, t1.refunds_returns = t2.refunds_returns, t1.refunds = t2.refunds, t1.plan_id = t2.plan_id, t1.plan_type = t2.plan_type, t1.time_created = t2.time_created, t1.payment_time = t2.payment_time, t1.time_ship = t2.time_ship, t1.time_delivered = t2.time_delivered, t1.time_completed = t2.time_completed, t1.time_commission_paid = t2.time_commission_paid, t1.brand = t2.brand
WHEN NOT MATCHED THEN
  INSERT (order_id, product_id, product_name, sku_sku, sku_id, seller_sku, price, payment_amount, currency, quantity, payment_method, order_status, creator_username, content_type, content_id, commission_rate_percentage, est_commision_base, est_commision_payment, act_commision_base, act_commision_payment, refunds_returns, refunds, plan_id, plan_type, time_created, payment_time, time_ship, time_delivered, time_completed, time_commission_paid, brand
)
  VALUES (order_id, product_id, product_name, sku_sku, sku_id, seller_sku, price, payment_amount, currency, quantity, payment_method, order_status, creator_username, content_type, content_id, commission_rate_percentage, est_commision_base, est_commision_payment, act_commision_base, act_commision_payment, refunds_returns, refunds, plan_id, plan_type, time_created, payment_time, time_ship, time_delivered, time_completed, time_commission_paid, brand
);
"""
project_id = "spmena-sandbox-vn-apac-dv"
pd.read_gbq(sql,project_id=project_id, use_bqstorage_api=True)  
print('Done merging')

Done merging


----------------------------------------------------------------------------------------------------------------------------

## 3. Livestream

### 3.A. Filter which csv files will be used for update

In [25]:
csv_files_raw = []
folder_lst = []

for division in division_lst:
    folder_path = folder_main + division
    folder_lst.append(folder_path)
for folder in folder_lst:
    folder_order = folder + "/Livestream/"
    csv_files_lst = glob.glob(folder_order + "\*.xlsx")
    for csv_file in csv_files_lst:
        csv_files_raw.append(csv_file)

In [26]:
csv_files = []

def extract_month(folder_location_string):
    date_extract_pattern = r'\d{4}_\d{2}_\d{2}'
    match = re.search(date_extract_pattern, folder_location_string)
    if match:
        extracted_date = match.group()
        date_formatted = int(extracted_date.replace("_", ""))
        if date_formatted >= date_formatted_input:
            csv_files.append(folder_location_string)  
    
for i in csv_files_raw:
    extract_month(i)   
    
csv_files    

["C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/CPD/Livestream\\mny_tiktok_livestream_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/CPD/Livestream\\oap_tiktok_livestream_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LDB/Livestream\\cerave_tiktok_livestream_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LDB/Livestream\\derma_tiktok_livestream_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LLD/Livestream\\kiehls_tiktok_livestream_2023_12_01.xlsx",
 "C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- Data Hub - Analytics/Tiktok Shop/LLD/Livestream\\kiehls_tiktok_livestream_2023_12_01_2023_12_18.xlsx"]

### 3.B. Append all csv files into a dataframe

In [27]:
dfs = []

for file in csv_files:
    df = pd.read_excel(file, dtype={
            'creator_id': 'object'
            ,'live_id': 'object'
            ,'creator_bind_shop_country': 'object'
        })
    
    if re.search("cerave", str(file), re.IGNORECASE):
        df['brand'] = 'CeraVe'
    elif re.search("kiehls", str(file), re.IGNORECASE):
        df['brand'] = "Kiehl's"
    elif re.search("oap", str(file), re.IGNORECASE):
        df['brand'] = "L'Oreal Paris"
    elif re.search("mny", str(file), re.IGNORECASE):
        df['brand'] = "Maybelline"
    elif re.search("derma", str(file), re.IGNORECASE):
        df['brand'] = "Derma"
        
    dfs.append(df)

df_livestream = pd.concat(dfs)
df_livestream.head(1)

,creator_id,creator_handle,creator_alias,creator_type,is_commissioned,creator_bind_shop_country,live_id,start_timestamp,end_timestamp,duration,revenue,avg_unit_price,products_cnt,products_with_sales_cnt,product_views_cnt,product_clicks_cnt,items_sold_cnt,buyers_cnt,orders_created_cnt,orders_paid_cnt,avg_concurrent_users_cnt,peak_concurrent_users_cnt,co_rate,local_avg_watch_duration,local_ctr_rate,local_new_followers_cnt,local_shares_cnt,local_comments_cnt,local_likes_cnt,local_views_cnt,local_viewers_cnt,date_scrape,brand
0,7164052841220621338,phapphuckhaianshop1,Pháp Phục Khải An,3,False,NaN,7307669338432654081,1701449443,1701455366,5923,0,0,1,0,45,5,0,0,0,0,10,75,0.0,49,0.0054,11,7,118,7754,927,846,2023-12-01,Maybelline


### 3.C. Clean the livestream dataframe

In [28]:
def timestamp_to_datetime(x): 
    return datetime.fromtimestamp(x)

df_livestream['start_timestamp'] = df_livestream['start_timestamp'].apply(timestamp_to_datetime)
df_livestream['end_timestamp'] = df_livestream['end_timestamp'].apply(timestamp_to_datetime)
df_livestream.head(1)

,creator_id,creator_handle,creator_alias,creator_type,is_commissioned,creator_bind_shop_country,live_id,start_timestamp,end_timestamp,duration,revenue,avg_unit_price,products_cnt,products_with_sales_cnt,product_views_cnt,product_clicks_cnt,items_sold_cnt,buyers_cnt,orders_created_cnt,orders_paid_cnt,avg_concurrent_users_cnt,peak_concurrent_users_cnt,co_rate,local_avg_watch_duration,local_ctr_rate,local_new_followers_cnt,local_shares_cnt,local_comments_cnt,local_likes_cnt,local_views_cnt,local_viewers_cnt,date_scrape,brand
0,7164052841220621338,phapphuckhaianshop1,Pháp Phục Khải An,3,False,NaN,7307669338432654081,2023-12-01 23:50:43,2023-12-02 01:29:26,5923,0,0,1,0,45,5,0,0,0,0,10,75,0.0,49,0.0054,11,7,118,7754,927,846,2023-12-01,Maybelline


### 3.D. Write livestream to GCP

A. Create a new table

In [29]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "spmena-sandbox-vn-apac-dv"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'tiktok_dashboard.tiktok_livestream_intraday'

pandas_gbq.to_gbq(df_livestream, table_id, project_id=project_id, if_exists = 'replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


B. Merge to full table

In [ ]:
sql = """
MERGE INTO `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_livestream` t1
USING `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_livestream_intraday` t2
ON t1.live_id = t2.live_id and t1.date_scrape = t2.date_scrape and t1.brand = t2.brand
when matched then
  update
  set t1.creator_id = t2.creator_id, t1.creator_handle = t2.creator_handle, t1.creator_alias = t2.creator_alias, t1.creator_type = t2.creator_type, t1.is_commissioned = t2.is_commissioned, t1.creator_bind_shop_country = t2.creator_bind_shop_country, t1.start_timestamp = t2.start_timestamp, t1.end_timestamp = t2.end_timestamp, t1.duration = t2.duration, t1.revenue = t2.revenue, t1.avg_unit_price = t2.avg_unit_price, t1.products_cnt = t2.products_cnt, t1.products_with_sales_cnt = t2.products_with_sales_cnt, t1.product_views_cnt = t2.product_views_cnt, t1.product_clicks_cnt = t2.product_clicks_cnt, t1.items_sold_cnt = t2.items_sold_cnt, t1.buyers_cnt = t2.buyers_cnt, t1.orders_created_cnt = t2.orders_created_cnt, t1.orders_paid_cnt = t2.orders_paid_cnt, t1.avg_concurrent_users_cnt = t2.avg_concurrent_users_cnt, t1.peak_concurrent_users_cnt = t2.peak_concurrent_users_cnt, t1.co_rate = t2.co_rate, t1.local_avg_watch_duration = t2.local_avg_watch_duration, t1.local_ctr_rate = t2.local_ctr_rate, t1.local_new_followers_cnt = t2.local_new_followers_cnt, t1.local_shares_cnt = t2.local_shares_cnt, t1.local_comments_cnt = t2.local_comments_cnt, t1.local_likes_cnt = t2.local_likes_cnt, t1.local_views_cnt = t2.local_views_cnt, t1.local_viewers_cnt = t2.local_viewers_cnt

WHEN NOT MATCHED THEN
  INSERT (creator_id, creator_handle, creator_alias, creator_type, is_commissioned, creator_bind_shop_country, live_id, start_timestamp, end_timestamp, duration, revenue, avg_unit_price, products_cnt, products_with_sales_cnt, product_views_cnt, product_clicks_cnt, items_sold_cnt, buyers_cnt, orders_created_cnt, orders_paid_cnt, avg_concurrent_users_cnt, peak_concurrent_users_cnt, co_rate, local_avg_watch_duration, local_ctr_rate, local_new_followers_cnt, local_shares_cnt, local_comments_cnt, local_likes_cnt, local_views_cnt, local_viewers_cnt, date_scrape, brand)
  VALUES (creator_id, creator_handle, creator_alias, creator_type, is_commissioned, creator_bind_shop_country, live_id, start_timestamp, end_timestamp, duration, revenue, avg_unit_price, products_cnt, products_with_sales_cnt, product_views_cnt, product_clicks_cnt, items_sold_cnt, buyers_cnt, orders_created_cnt, orders_paid_cnt, avg_concurrent_users_cnt, peak_concurrent_users_cnt, co_rate, local_avg_watch_duration, local_ctr_rate, local_new_followers_cnt, local_shares_cnt, local_comments_cnt, local_likes_cnt, local_views_cnt, local_viewers_cnt, date_scrape, brand);
"""
project_id = "spmena-sandbox-vn-apac-dv"
pd.read_gbq(sql,project_id=project_id, use_bqstorage_api=True)    
print('Done merging')

----------------------------------------------------------------------------------------------------------------------------

## 4. Video

### 4.A. Filter which csv files will be used for update

In [ ]:
csv_files_raw = []
folder_lst = []

for division in division_lst:
    folder_path = folder_main + division
    folder_lst.append(folder_path)
for folder in folder_lst:
    folder_order = folder + "/Video/"
    csv_files_lst = glob.glob(folder_order + "\*.xlsx")
    for csv_file in csv_files_lst:
        csv_files_raw.append(csv_file)

In [ ]:
csv_files = []

def extract_month(folder_location_string):
    date_extract_pattern = r'\d{4}_\d{2}_\d{2}'
    match = re.search(date_extract_pattern, folder_location_string)
    if match:
        extracted_date = match.group()
        date_formatted = int(extracted_date.replace("_", ""))
        if date_formatted >= date_formatted_input:
            csv_files.append(folder_location_string)  
    
for i in csv_files_raw:
    extract_month(i)   
    
csv_files    

### 4.B. Append all csv files into a dataframe

In [ ]:
dfs = []

for file in csv_files:
    df = pd.read_excel(file, dtype={
            'video_id': 'string'
            ,'video_name': 'string'
            ,'creator_id': 'string'
            ,'video_publish_time': 'string'
            ,'video_url': 'string'
            ,'creator_handle': 'string'
            ,'creator_alias': 'string'
            ,'view_cnt': 'int64'
            ,'like_cnt': 'int64'
            ,'comment_cnt': 'int64'
            ,'share_cnt': 'int64'
            ,'new_follower_cnt': 'int64'
            ,'item_click_cnt': 'int64'
            ,'item_exposure_cnt': 'int64'
            ,'item_sold_cnt': 'int64'
            ,'item_buyer_cnt': 'int64'
            ,'item_click_cnt': 'int64'
            ,'revenue': 'int64'
            ,'refunds': 'int64'
            ,'commission_estimated': 'int64'
            ,'product_refund_item_cnt': 'int64'
            ,'sku_order_created_cnt': 'int64'
    })
    
    if re.search("cerave", str(file), re.IGNORECASE):
        df['brand'] = 'CeraVe'
    elif re.search("kiehls", str(file), re.IGNORECASE):
        df['brand'] = "Kiehl's"
    elif re.search("oap", str(file), re.IGNORECASE):
        df['brand'] = "L'Oreal Paris"
    elif re.search("mny", str(file), re.IGNORECASE):
        df['brand'] = "Maybelline"
    elif re.search("derma", str(file), re.IGNORECASE):
        df['brand'] = "Derma"
        
    dfs.append(df)

df_video = pd.concat(dfs)
df_video.head(1)

### 4.C. Clean the video dataframe

In [ ]:
def timestamp_to_datetime_vid(x): 
    try:
        return datetime.fromtimestamp(int(x))
    except: return None
    
df_video['video_publish_time'] = df_video['video_publish_time'].apply(timestamp_to_datetime_vid)

if 'building_block' in df_video.columns:
    df_video.drop('building_block', axis=1, inplace=True)

df_video.head(1)

### 4.D. Write video to GCP

A. Create a new table

In [ ]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "spmena-sandbox-vn-apac-dv"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'tiktok_dashboard.tiktok_video_intraday'

pandas_gbq.to_gbq(df_video, table_id, project_id=project_id, if_exists = 'replace')

B. Merge to full table

In [ ]:
sql = """
MERGE INTO `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_video` t1
USING `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_video_intraday` t2
ON t1.video_id = t2.video_id and t1.date_scrape = t2.date_scrape and t1.brand = t2.brand

WHEN NOT MATCHED THEN
  INSERT (video_id, video_name, video_url, video_publish_time, duration, creator_id, creator_handle, creator_alias, view_cnt, like_cnt, comment_cnt, share_cnt, new_follower_cnt, item_exposure_cnt, item_click_cnt, item_sold_cnt, item_buyer_cnt, revenue, refunds, commission_estimated, product_refund_item_cnt, co_rate, sku_order_created_cnt, ctr_rate, date_scrape, brand)
  VALUES (video_id, video_name, video_url, video_publish_time, duration, creator_id, creator_handle, creator_alias, view_cnt, like_cnt, comment_cnt, share_cnt, new_follower_cnt, item_exposure_cnt, item_click_cnt, item_sold_cnt, item_buyer_cnt, revenue, refunds, commission_estimated, product_refund_item_cnt, co_rate, sku_order_created_cnt, ctr_rate, date_scrape, brand);
"""
project_id = "spmena-sandbox-vn-apac-dv"
pd.read_gbq(sql,project_id=project_id, use_bqstorage_api=True)    
print('Done merging')

----------------------------------------------------------------------------------------------------------------------------

## 5. Product

### 5.A. Filter which csv files will be used for update

In [ ]:
csv_files_raw = []
folder_lst = []

for division in division_lst:
    folder_path = folder_main + division
    folder_lst.append(folder_path)
for folder in folder_lst:
    folder_order = folder + "/Product/"
    csv_files_lst = glob.glob(folder_order + "\*.xlsx")
    for csv_file in csv_files_lst:
        csv_files_raw.append(csv_file)

In [ ]:
csv_files = []

def extract_month(folder_location_string):
    date_extract_pattern = r'\d{4}_\d{2}_\d{2}'
    match = re.search(date_extract_pattern, folder_location_string)
    if match:
        extracted_date = match.group()
        date_formatted = int(extracted_date.replace("_", ""))
        if date_formatted >= date_formatted_input:
            csv_files.append(folder_location_string)  
    
for i in csv_files_raw:
    extract_month(i)   
    
csv_files    

### 5.B. Append all csv files into a dataframe

In [ ]:
dfs = []

for file in csv_files:
    df = pd.read_excel(file, dtype={
            'product_id': 'object'
            ,'revenue': 'float'
            # ,'creator_bind_shop_country': 'object'
    })
    
    if re.search("cerave", str(file), re.IGNORECASE):
        df['brand'] = 'CeraVe'
    elif re.search("kiehls", str(file), re.IGNORECASE):
        df['brand'] = "Kiehl's"
    elif re.search("oap", str(file), re.IGNORECASE):
        df['brand'] = "L'Oreal Paris"
    elif re.search("mny", str(file), re.IGNORECASE):
        df['brand'] = "Maybelline"
    elif re.search("derma", str(file), re.IGNORECASE):
        df['brand'] = "Derma"
        
    dfs.append(df)

df_product = pd.concat(dfs)
if 'is_competitive_price' in df_product.columns:
    df_product = df_product.drop('is_competitive_price', axis=1)
df_product.head(1)

### 5.C. Write product to GCP

A. Create a new table

In [ ]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "spmena-sandbox-vn-apac-dv"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'tiktok_dashboard.tiktok_product_intraday'

pandas_gbq.to_gbq(df_product, table_id, project_id=project_id, if_exists = 'replace')

B. Merge to full table

In [ ]:
sql = """
MERGE INTO `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_product` t1
USING `spmena-sandbox-vn-apac-dv.tiktok_dashboard.tiktok_product_intraday` t2
ON t1.product_id = t2.product_id and t1.date_scrape = t2.date_scrape

WHEN NOT MATCHED THEN
  INSERT (product_id, shop_id, product_name, product_image, revenue, revenue_live, revenue_video, buyer_cnt, item_sold_cnt, sku_order_cnt, item_sold_live_cnt, buyer_live_cnt, viewer_live_cnt, unique_item_click_live_cnt, product_view_cnt_live, product_click_cnt_live, item_sold_video_cnt, sku_order_video_cnt, buyer_video_cnt, viewer_video_cnt, unique_item_click_video_cnt, product_view_cnt_video, product_click_cnt_video, product_co_rate_live, product_ctr_rate_live, product_co_rate_video, product_ctr_rate_video, date_scrape, brand)
  VALUES (product_id, shop_id, product_name, product_image, revenue, revenue_live, revenue_video, buyer_cnt, item_sold_cnt, sku_order_cnt, item_sold_live_cnt, buyer_live_cnt, viewer_live_cnt, unique_item_click_live_cnt, product_view_cnt_live, product_click_cnt_live, item_sold_video_cnt, sku_order_video_cnt, buyer_video_cnt, viewer_video_cnt, unique_item_click_video_cnt, product_view_cnt_video, product_click_cnt_video, product_co_rate_live, product_ctr_rate_live, product_co_rate_video, product_ctr_rate_video, date_scrape, brand)
"""
project_id = "spmena-sandbox-vn-apac-dv"
pd.read_gbq(sql,project_id=project_id, use_bqstorage_api=True)    
print('Done merging')

----------------------------------------------------------------------------------------------------------------------------

## 6. Product dim table (LDB)

In [ ]:
df_brand_mapping_ldb = pd.read_excel("C:/Users/gianhubinh.le/OneDrive - L'Oréal/Socom _ Daily Performance/bundle_data.xlsx", sheet_name = 'Sheet1')
df_brand_mapping_ldb = df_brand_mapping_ldb[['Brand','Barcode','Component barcode']]
df_brand_mapping_ldb = df_brand_mapping_ldb.drop_duplicates()
df_brand_mapping_ldb = df_brand_mapping_ldb.dropna(subset=['Brand'])
df_brand_mapping_ldb = df_brand_mapping_ldb.rename(columns={'Component barcode': 'Component_barcode'})
df_brand_mapping_ldb.columns = df_brand_mapping_ldb.columns.str.lower()
df_brand_mapping_ldb.head(2)

In [ ]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "spmena-sandbox-vn-apac-dv"

# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'tiktok_dashboard.product_mapping_ldb'

pandas_gbq.to_gbq(df_brand_mapping_ldb, table_id, project_id=project_id, if_exists = 'replace')